# 随机梯度下降 (Stochastic Gradient Descent)

---

## 📚 学习目标

通过本教程，你将学会：
- 理解随机梯度下降与批量梯度下降的区别
- 实现SGD和Mini-batch GD
- 理解学习率衰减策略
- 使用Scikit-learn的SGDRegressor

## 📋 前置知识

- 批量梯度下降
- 线性回归

## ⏱️ 预计时间

25-35分钟

---

## 📖 第1部分：理论背景

### 梯度下降的三种变体

| 类型 | 每次使用的样本数 | 特点 |
|------|----------------|------|
| 批量GD | 全部 (m) | 稳定但慢，内存需求大 |
| 随机GD | 1个 | 快但不稳定，可逃离局部最优 |
| 小批量GD | b个 (1<b<m) | 平衡速度和稳定性 |

### 随机梯度下降的优点

1. **更快的迭代**: 每次只用一个样本
2. **可处理大数据**: 不需要加载全部数据
3. **可逃离局部最优**: 随机性带来的优势
4. **在线学习**: 可实时更新模型

### 学习率衰减

由于SGD的随机性，需要逐渐降低学习率以确保收敛：

$$\eta(t) = \frac{\eta_0}{1 + t/t_1}$$

> 💡 **关键洞察**: SGD的随机性是一把双刃剑——帮助逃离局部最优，但也导致收敛不稳定。

---

## 💻 第2部分：代码实现

### 步骤1: 准备工作

In [ ]:
# ============================================================
# 导入库和生成数据
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 生成数据
m = 100
X = 2 * np.random.rand(m, 1)
y = 4 + 3 * X + np.random.randn(m, 1) * 0.5
X_b = np.c_[np.ones((m, 1)), X]

TRUE_THETA_0, TRUE_THETA_1 = 4.0, 3.0
print("✓ 数据准备完成")

### 步骤2: 实现随机梯度下降

In [ ]:
# ============================================================
# 随机梯度下降实现
# ============================================================

def learning_schedule(t, t0=5, t1=50):
    """学习率衰减函数"""
    return t0 / (t + t1)

def stochastic_gradient_descent(X_b, y, n_epochs=50, t0=5, t1=50, record_history=False):
    """
    随机梯度下降
    
    参数:
        X_b: 带偏置的特征矩阵
        y: 目标值
        n_epochs: 迭代轮数
        t0, t1: 学习率衰减参数
        record_history: 是否记录历史
    """
    m = len(y)
    theta = np.random.randn(2, 1)
    
    history = {'theta': [theta.copy()], 'cost': []}
    
    for epoch in range(n_epochs):
        # 每个epoch随机打乱数据顺序
        shuffled_indices = np.random.permutation(m)
        
        for i in range(m):
            # 随机选取一个样本
            random_index = shuffled_indices[i]
            xi = X_b[random_index:random_index+1]
            yi = y[random_index:random_index+1]
            
            # 计算梯度（单样本）
            gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
            
            # 计算当前学习率
            eta = learning_schedule(epoch * m + i, t0, t1)
            
            # 更新参数
            theta = theta - eta * gradients
        
        if record_history:
            history['theta'].append(theta.copy())
            predictions = X_b.dot(theta)
            cost = (1/(2*m)) * np.sum((predictions - y)**2)
            history['cost'].append(cost)
    
    if record_history:
        return theta, history
    return theta

print("✓ SGD函数定义完成")

In [ ]:
# ============================================================
# 运行SGD
# ============================================================

theta_sgd, history_sgd = stochastic_gradient_descent(
    X_b, y, n_epochs=50, record_history=True
)

print("SGD结果:")
print(f"  θ₀: {theta_sgd[0][0]:.6f} (真实: {TRUE_THETA_0})")
print(f"  θ₁: {theta_sgd[1][0]:.6f} (真实: {TRUE_THETA_1})")

### 步骤3: 实现Mini-batch梯度下降

In [ ]:
# ============================================================
# Mini-batch梯度下降
# ============================================================

def minibatch_gradient_descent(X_b, y, n_epochs=50, batch_size=16, 
                               eta=0.1, record_history=False):
    """
    Mini-batch梯度下降
    
    参数:
        batch_size: 每批样本数
    """
    m = len(y)
    theta = np.random.randn(2, 1)
    
    history = {'theta': [theta.copy()], 'cost': []}
    n_batches = int(np.ceil(m / batch_size))
    
    for epoch in range(n_epochs):
        shuffled_indices = np.random.permutation(m)
        X_b_shuffled = X_b[shuffled_indices]
        y_shuffled = y[shuffled_indices]
        
        for i in range(n_batches):
            start = i * batch_size
            end = min(start + batch_size, m)
            
            X_batch = X_b_shuffled[start:end]
            y_batch = y_shuffled[start:end]
            
            # 计算梯度（批量）
            gradients = (2/len(y_batch)) * X_batch.T.dot(X_batch.dot(theta) - y_batch)
            theta = theta - eta * gradients
        
        if record_history:
            history['theta'].append(theta.copy())
            cost = (1/(2*m)) * np.sum((X_b.dot(theta) - y)**2)
            history['cost'].append(cost)
    
    if record_history:
        return theta, history
    return theta

theta_mb, history_mb = minibatch_gradient_descent(
    X_b, y, n_epochs=50, batch_size=16, record_history=True
)

print("Mini-batch GD结果:")
print(f"  θ₀: {theta_mb[0][0]:.6f}")
print(f"  θ₁: {theta_mb[1][0]:.6f}")

---

## 📊 第3部分：三种方法对比

In [ ]:
# ============================================================
# 批量GD用于对比
# ============================================================

def batch_gradient_descent(X_b, y, n_epochs=50, eta=0.1, record_history=False):
    m = len(y)
    theta = np.random.randn(2, 1)
    history = {'theta': [theta.copy()], 'cost': []}
    
    for epoch in range(n_epochs):
        gradients = (2/m) * X_b.T.dot(X_b.dot(theta) - y)
        theta = theta - eta * gradients
        
        if record_history:
            history['theta'].append(theta.copy())
            cost = (1/(2*m)) * np.sum((X_b.dot(theta) - y)**2)
            history['cost'].append(cost)
    
    if record_history:
        return theta, history
    return theta

# 运行批量GD
np.random.seed(42)
_, history_bgd = batch_gradient_descent(X_b, y, n_epochs=50, record_history=True)

# 重新运行SGD和Mini-batch确保相同初始化
np.random.seed(42)
_, history_sgd = stochastic_gradient_descent(X_b, y, n_epochs=50, record_history=True)

np.random.seed(42)
_, history_mb = minibatch_gradient_descent(X_b, y, n_epochs=50, batch_size=16, record_history=True)

In [ ]:
# ============================================================
# 可视化对比
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 损失曲线对比
axes[0].plot(history_bgd['cost'], 'b-', linewidth=2, label='批量GD')
axes[0].plot(history_sgd['cost'], 'r-', linewidth=2, alpha=0.7, label='随机GD')
axes[0].plot(history_mb['cost'], 'g-', linewidth=2, alpha=0.7, label='Mini-batch GD')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('损失')
axes[0].set_title('三种梯度下降方法对比', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 参数空间路径
theta0_range = np.linspace(2, 6, 50)
theta1_range = np.linspace(1, 5, 50)
theta0_grid, theta1_grid = np.meshgrid(theta0_range, theta1_range)
cost_grid = np.zeros_like(theta0_grid)

for i in range(len(theta0_range)):
    for j in range(len(theta1_range)):
        theta_temp = np.array([[theta0_range[i]], [theta1_range[j]]])
        cost_grid[j, i] = (1/(2*m)) * np.sum((X_b.dot(theta_temp) - y)**2)

axes[1].contour(theta0_grid, theta1_grid, cost_grid, levels=20, cmap='viridis')

# 绘制路径
for hist, color, label in [(history_bgd, 'b', '批量GD'), 
                           (history_sgd, 'r', '随机GD'),
                           (history_mb, 'g', 'Mini-batch')]:
    theta_hist = np.array(hist['theta']).squeeze()
    axes[1].plot(theta_hist[:, 0], theta_hist[:, 1], f'{color}.-', 
                linewidth=1, markersize=3, alpha=0.7, label=label)

axes[1].scatter(TRUE_THETA_0, TRUE_THETA_1, c='black', s=100, marker='*', 
               zorder=5, label='最优点')
axes[1].set_xlabel('θ₀')
axes[1].set_ylabel('θ₁')
axes[1].set_title('参数空间中的优化路径', fontweight='bold')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n观察结论:")
print("  - 批量GD: 路径平滑，直接走向最优")
print("  - 随机GD: 路径震荡，但最终收敛")
print("  - Mini-batch: 介于两者之间")

---

## 🔧 第4部分：使用Scikit-learn

In [ ]:
# ============================================================
# 使用sklearn的SGDRegressor
# ============================================================

sgd_reg = SGDRegressor(
    max_iter=1000,          # 最大迭代次数
    tol=1e-3,               # 收敛阈值
    penalty=None,           # 无正则化
    eta0=0.1,               # 初始学习率
    learning_rate='invscaling',  # 学习率衰减策略
    random_state=RANDOM_SEED
)

sgd_reg.fit(X, y.ravel())

print("sklearn SGDRegressor结果:")
print(f"  θ₀: {sgd_reg.intercept_[0]:.6f}")
print(f"  θ₁: {sgd_reg.coef_[0]:.6f}")

print(f"\n真实值:")
print(f"  θ₀: {TRUE_THETA_0}")
print(f"  θ₁: {TRUE_THETA_1}")

---

## 📝 总结

### 关键要点

1. ✅ **SGD**每次只用一个样本更新，速度快但不稳定
2. ✅ **Mini-batch GD**平衡了速度和稳定性
3. ✅ **学习率衰减**帮助SGD稳定收敛
4. ✅ 在大数据集上，SGD和Mini-batch更实用

### 方法选择指南

| 场景 | 推荐方法 |
|------|----------|
| 小数据集 | 批量GD |
| 大数据集 | Mini-batch GD |
| 在线学习 | SGD |
| GPU训练 | Mini-batch GD |

### 下一步学习

- 📘 动量法 (Momentum)
- 📙 Adam优化器
- 📕 学习率调度策略

## 📚 参考资料

- [Scikit-learn: SGDRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html)
- Bottou, L. (2010). "Large-Scale Machine Learning with Stochastic Gradient Descent"